In [1]:
import matplotlib.pyplot as plt
import pandas
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
import json
import os
import deepsig
from IPython.display import display

2023-09-19 22:00:08.501884: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 22:00:08.533372: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 22:00:08.533909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 22:00:09.372252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def convert_to_json_of_arrays(array_of_jsons):
    json_of_arrays = {}

    # Iterate through each JSON in the array
    for json_obj in array_of_jsons:
        for key, value in json_obj.items():
            # Check if the key exists in the new JSON, if not, create an array
            if key not in json_of_arrays:
                json_of_arrays[key] = []

            # Append the value to the corresponding array
            json_of_arrays[key].append(value)

    return json_of_arrays

In [3]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [4]:
def read_json_files_from_folder(folder_path):
    json_list = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .json extension
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)

            # Read the contents of the JSON file
            with open(file_path, "r") as file:
                try:
                    # Parse the JSON data and append it to the list
                    json_data = json.load(file)
                    json_list.append(json_data)
                except json.JSONDecodeError:
                    # Handle parsing errors, if any
                    print(f"Error parsing JSON from file: {file_name}")

    return json_list

In [5]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [6]:
results = read_csv_files_from_folder('../2023-07-26/results')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [7]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

In [8]:
display(results)

,Unnamed: 0,overall_acc,bal_acc,Equalized Odds,disp_imp,Statistical Parity,Equal Opportunity,theil_ind,f1_score,TPR,...,solution,tune_results_history,fitness_rule,method,dataset,best_solution_tf_history,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,0,0.650000,0.477941,0.022839,1.007030,0.006558,0.028241,0.087680,0.804954,0.955882,...,{'dropout': 0.18124389809074126},"[{'overall_acc': 0.59375, 'bal_acc': 0.4481132...",mcc_parity,Standard MLP (baseline),German Credit,"{'loss': [0.7094872196515402, 0.64226885371738...",0.000000,0.006558,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,1,0.645000,0.484962,0.073451,1.064832,0.059217,0.045977,0.077539,0.806250,0.969925,...,"{'dropout': 0.022732599406043996, 'privileged_...","[{'overall_acc': 0.00625, 'bal_acc': 0.0043859...",mcc_parity,Fair Transition Loss,German Credit,"{'loss': [0.8503885070482889, 0.78124442100524...",0.000000,0.059217,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
2,2,0.740000,0.659133,0.025781,0.954128,0.038462,0.017416,0.092947,0.823129,0.937984,...,{'adversary_loss_weight': 0.6148404367300289},"[{'overall_acc': 0.70625, 'bal_acc': 0.5580357...",mcc_parity,Adversarial Debiasing,German Credit,NaN,0.400810,0.038462,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
3,3,0.545000,0.397810,0.070884,0.895425,0.077859,0.056402,0.199996,0.775801,0.795620,...,{'eta': 49.471477392162484},"[{'overall_acc': 0.56875, 'bal_acc': 0.4136363...",mcc_parity,Prejudice Remover,German Credit,NaN,0.000000,0.077859,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
4,4,0.615000,0.448905,0.066751,0.894180,0.087912,0.050342,0.121159,0.828283,0.897810,...,{'dropout': 0.0037868792501197348},"[{'overall_acc': 0.56875, 'bal_acc': 0.4099099...",mcc_odds,Standard MLP (baseline),German Credit,"{'loss': [0.7087434199121263, 0.61068538427352...",0.000000,0.066751,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,25,0.896851,0.669304,0.057653,2.118719,0.074544,0.087314,0.093049,0.472758,0.364341,...,"{'C': 10.459577021264977, 'gamma': 0.1}","[{'overall_acc': 0.8993439934399344, 'bal_acc'...",acc_opportunity,Gerry Fair Classifier,Bank Marketing,NaN,0.896851,0.087314,Max(Accuracy - Equal Opportunity),Max(Acc - Eq. Odds)
1753,26,0.896359,0.753549,0.072641,2.153847,0.133189,0.120516,0.075922,0.584757,0.560453,...,{'dropout': 0.000321942353699714},"[{'overall_acc': 0.9093890938909389, 'bal_acc'...",acc_opportunity,Standard MLP (baseline),Bank Marketing,"{'loss': [0.2854798949427075, 0.22084286485976...",0.896359,0.120516,Max(Accuracy - Equal Opportunity),Max(Acc - Eq. Odds)
1754,27,0.898655,0.696844,0.053642,1.420554,0.034509,0.099174,0.086198,0.514914,0.427083,...,"{'dropout': 0.014472416047403187, 'privileged_...","[{'overall_acc': 0.8511685116851169, 'bal_acc'...",acc_opportunity,Fair Transition Loss,Bank Marketing,"{'loss': [0.49211953761153904, 0.4700222834873...",0.898655,0.099174,Max(Accuracy - Equal Opportunity),Max(Acc - Eq. Odds)
1755,28,0.901607,0.718345,0.145878,0.755483,0.022450,0.286523,0.081914,0.551570,0.471867,...,{'adversary_loss_weight': 0.8391213408528397},"[{'overall_acc': 0.8987289872898729, 'bal_acc'...",acc_opportunity,Adversarial Debiasing,Bank Marketing,NaN,0.901607,0.286523,Max(Accuracy - Equal Opportunity),Max(Acc - Eq. Odds)


In [9]:
datasets = results['dataset'].unique().tolist()
datasets

['German Credit', 'Bank Marketing', 'Adult Income', 'Compas Recidivism']

In [10]:
fitness_rules = results['fitness_rule'].unique().tolist()
fitness_rules

['mcc_parity',
 'mcc_odds',
 'mcc_opportunity',
 'acc_parity',
 'acc_odds',
 'acc_opportunity']

In [11]:
methods = results['method'].unique().tolist()
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']

In [12]:
if os.path.exists('multi_aso_data_list.json'):
    with open('multi_aso_data_list.json') as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open('multi_aso_data_list.json', 'w') as file:
        json.dump(multi_aso_data_list, file)

In [13]:
multi_aso_data_list

[{'fitness_rule': 'mcc_parity',
  'dataset': 'Adult Income',
  'min_eps': [[1.0,
    0.9988805908520648,
    0.9979750223790821,
    0.995877444722284,
    0.0],
   [0.004620261792774651, 1.0, 0.1528664058142362, 1.0, 0.0],
   [0.004940540275507269, 1.0, 1.0, 1.0, 0.0],
   [0.00595911008417298, 1.0, 0.1050132284790311, 1.0, 0.0],
   [1.0, 1.0, 1.0, 1.0, 1.0]]},
 {'fitness_rule': 'mcc_odds',
  'dataset': 'Adult Income',
  'min_eps': [[1.0, 1.0, 1.0, 1.0, 0.0],
   [0.027617628263128934, 1.0, 0.25458170757222737, 0.42157426878629345, 0.0],
   [0.251460472570234, 1.0, 1.0, 0.7790530472236021, 0.0],
   [0.8744194609335025, 1.0, 1.0, 1.0, 0.04857425621287411],
   [1.0, 1.0, 1.0, 1.0, 1.0]]},
 {'fitness_rule': 'mcc_opportunity',
  'dataset': 'Adult Income',
  'min_eps': [[1.0,
    0.9976536340274126,
    0.9789694086170047,
    0.8910534937566084,
    0.0],
   [0.008614325158317561,
    1.0,
    0.0032188112253179217,
    0.058227525767044014,
    0.0],
   [1.0, 1.0, 1.0, 0.9400209938187005, 

In [17]:
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']
    metrics = fitness_rules_target_metrics[fitness_rule]
    subset_results = results.loc[ (results.fitness_rule == fitness_rule) \
                                      & (results.dataset == dataset) ]

    grouped_results = subset_results\
        .groupby(['method'])\
        .agg({'fitness': ['mean', 'std','count'], metrics[0]: ['mean', 'std'], metrics[1]: ['mean', 'std']})\
        .sort_values(by=('fitness','mean'), ascending=False)

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['methods'] = methods
    aso_df = aso_df.set_index('methods')

    print('%s | max(%s - %s)' % (dataset, metrics[0], metrics[1]))
    display(grouped_results)
    print('Significance Testing')
    display(aso_df)

Adult Income | max(Mathew Correlation - Statistical Parity)


fitness                 Mathew Correlation  \
                             mean       std count               mean   
method                                                                 
Fair Transition Loss     0.492021  0.016387    16           0.512027   
Prejudice Remover        0.490575  0.008865    16           0.500024   
Adversarial Debiasing    0.477586  0.014882    16           0.501430   
Standard MLP (baseline)  0.395355  0.012556    17           0.580617   
Gerry Fair Classifier    0.304991  0.128019    16           0.453065   

                                  Statistical Parity            
                              std               mean       std  
method                                                          
Fair Transition Loss     0.010737           0.020006  0.010913  
Prejudice Remover        0.008227           0.009449  0.009857  
Adversarial Debiasing    0.020095           0.023844  0.022968  
Standard MLP (baseline)  0.008613           0.185263  0.010419  
Gerry Fair Classifier    0.173689           0.148074  0.080770

Significance Testing


,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.000000,0.998881,0.997975,0.995877,0.0
Fair Transition Loss,0.004620,1.000000,0.152866,1.000000,0.0
Adversarial Debiasing,0.004941,1.000000,1.000000,1.000000,0.0
Prejudice Remover,0.005959,1.000000,0.105013,1.000000,0.0
Gerry Fair Classifier,1.000000,1.000000,1.000000,1.000000,1.0


Adult Income | max(Mathew Correlation - Equalized Odds)


...

,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.0,0.739006,0.185844,0.265305,0.0
Fair Transition Loss,1.0,1.000000,0.429356,0.969500,0.0
Adversarial Debiasing,1.0,1.000000,1.000000,1.000000,0.0
Prejudice Remover,1.0,0.855692,0.286850,1.000000,0.0
Gerry Fair Classifier,1.0,1.000000,1.000000,1.000000,1.0


German Credit | max(Accuracy - Equal Opportunity)


fitness                  Accuracy            \
                             mean       std count      mean       std   
method                                                                  
Fair Transition Loss     0.679803  0.048559    15  0.692333  0.032616   
Standard MLP (baseline)  0.549810  0.059112    15  0.614333  0.033481   
Adversarial Debiasing    0.530106  0.328938    15  0.712667  0.099837   
Prejudice Remover        0.504640  0.082283    15  0.590000  0.025284   

                        Equal Opportunity            
                                     mean       std  
method                                               
Fair Transition Loss             0.012531  0.024006  
Standard MLP (baseline)          0.064523  0.045405  
Adversarial Debiasing            0.182560  0.235414  
Prejudice Remover                0.085360  0.074077

Significance Testing


,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.000000,0.998447,0.687304,0.298358,0.0
Fair Transition Loss,0.014195,1.000000,0.082456,0.000000,0.0
Adversarial Debiasing,1.000000,1.000000,1.000000,1.000000,0.0
Prejudice Remover,1.000000,1.000000,0.828166,1.000000,0.0
Gerry Fair Classifier,1.000000,1.000000,1.000000,1.000000,1.0


In [18]:
aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df = aso_df[aso_df['method'] == 'Fair Transition Loss' ]
    aso_df = aso_df.drop(['Fair Transition Loss'], axis=1)
    aso_df = aso_df.drop(['method'], axis=1)
    aso_df_resume.append(aso_df)


print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'])
display(significance)
significance.to_latex('significance_resume.tex')

Significance Testing


Standard MLP (baseline)  \
fitness_rule    dataset                                      
acc_odds        Adult Income                      0.009395   
                Bank Market                       0.775530   
                Compas Recidivism                 0.006612   
                German Credit                     1.000000   
acc_opportunity Adult Income                      0.898703   
                Bank Market                       1.000000   
                Compas Recidivism                 0.009027   
                German Credit                     0.014195   
acc_parity      Adult Income                      0.006226   
                Bank Market                       0.235047   
                Compas Recidivism                 0.002008   
                German Credit                     1.000000   
mcc_odds        Adult Income                      0.027618   
                Bank Market                       0.471688   
                Compas Recidivism                 0.005317   
                German Credit                     1.000000   
mcc_opportunity Adult Income                      0.008614   
                Bank Market                       0.820683   
                Compas Recidivism                 0.004837   
                German Credit                     1.000000   
mcc_parity      Adult Income                      0.004620   
                Bank Market                       0.008002   
                Compas Recidivism                 0.005095   
                German Credit                     0.362420   

                                   Adversarial Debiasing  Prejudice Remover  \
fitness_rule    dataset                                                       
acc_odds        Adult Income                    0.225552           0.170713   
                Bank Market                     0.386310           0.814048   
                Compas Recidivism               0.471443           0.000000   
                German Credit                   0.429356           0.969500   
acc_opportunity Adult Income                    0.959536           1.000000   
                Bank Market                     0.369527           0.819028   
                Compas Recidivism               0.792935           0.000000   
                German Credit                   0.082456           0.000000   
acc_parity      Adult Income                    0.252930           0.305378   
                Bank Market                     1.000000           1.000000   
                Compas Recidivism               1.000000           0.107223   
                German Credit                   0.469207           1.000000   
mcc_odds        Adult Income                    0.254582           0.421574   
                Bank Market                     0.189642           0.108138   
                Compas Recidivism               0.582610           0.000000   
                German Credit                   1.000000           1.000000   
mcc_opportunity Adult Income                    0.003219           0.058228   
                Bank Market                     0.180293           0.236480   
                Compas Recidivism               1.000000           0.000000   
                German Credit                   1.000000           1.000000   
mcc_parity      Adult Income                    0.152866           1.000000   
                Bank Market                     0.000000           0.000000   
                Compas Recidivism               0.211472           0.000000   
                German Credit                   1.000000           0.475996   

                                   Gerry Fair Classifier  
fitness_rule    dataset                                   
acc_odds        Adult Income                    0.000000  
                Bank Market                     1.000000  
                Compas Recidivism               0.001646  
                German Credit                   0.000000  
acc_opportuni

In [19]:
grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'mean')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'mean')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'mean')]:.2f})$", axis=1)
grouped_results

fitness  \
                                                                       mean   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.200070   
                                          Fair Transition Loss    -0.034435   
                                          Prejudice Remover       -0.082559   
                                          Standard MLP (baseline) -0.090370   
                        Compas Recidivism Fair Transition Loss     0.219648   
...                                                                     ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover        0.809534   
                                          Fair Transition Loss     0.787013   
                                          Adversarial Debiasing    0.756054   
                                          Standard MLP (baseline)  0.752260   
                                          Gerry Fair Classifier    0.655568   

                                                                             \
                                                                        std   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.172675   
                                          Fair Transition Loss     0.073668   
                                          Prejudice Remover        0.050746   
                                          Standard MLP (baseline)  0.061469   
                        Compas Recidivism Fair Transition Loss     0.060505   
...                                                                     ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover        0.022701   
                                          Fair Transition Loss     0.083934   
                                          Adversarial Debiasing    0.028345   
                                          Standard MLP (baseline)  0.041790   
                                          Gerry Fair Classifier    0.185910   

                                                                  Performance  \
                                                                         mean   
Fitness Rule Abvr       dataset           method                                
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing      0.367673   
                                          Fair Transition Loss       0.000000   
                                          Prejudice Remover          0.000000   
                                          Standard MLP (baseline)    0.000000   
                        Compas Recidivism Fair Transition Loss       0.276319   
...                                                                       ...   
Max(Acc - Eq. Odds)     Adult Income      Prejudice Remover          0.845788   
                                          Fair Transition Loss       0.825826   
                                          Adversarial Debiasing      0.847730   
                                          Standard MLP (baseline)    0.849309   
                                          Gerry Fair Classifier      0.720083   

                                                                             \
                                                                        std   
Fitness Rule Abvr       dataset           method                              
Max(MCC - Stat. Parity) German Credit     Adversarial Debiasing    0.056485   
                                          Fair Transition Loss     0.000000   
                                          Prejudice Remover        0.000000   
                                          Standard MLP (baseline)  0.000000   
                        Compas Recidivism Fair Transition Loss     0.027969   
...                                                                     ...   
Max(Acc 

In [20]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
grouped_results[selected_columns].to_latex('grouped_results.tex')